<a href="https://colab.research.google.com/github/JatekRaim/hakaton-case/blob/main/case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd

from pprint import pprint


In [38]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [39]:
dat_train = drive.CreateFile({'id':"1Oar4L1wMZ9_2KSfJGEs7HT83mfA_1k-u"})
dat_train.GetContentFile('train_dataset_Датасет.xlsx')
dat_train = pd.read_excel('train_dataset_Датасет.xlsx')
dat_train[:100]

,QUESTION,ANSWER
0,Какие основания для получения 33 услуги?,Заявителем является родитель (усыновитель) реб...
1,Какие основания для получения предоставления д...,Заявителем является родитель (усыновитель) реб...
2,Какие основания для получения единовременной к...,Заявителем является родитель (усыновитель) реб...
3,Какие основания для детской карты?,Заявителем является родитель (усыновитель) реб...
4,Какие условия являются обязательными для получ...,"ребенок - гражданин РФ, имеющий место жительст..."
...,...,...
95,Как подтвердить оплату ЖКХ при оформлении услу...,"Документы, подтверждающие начисление платы за ..."
96,Как подтвердить оплату ЖКХ при оформлении услу...,"Документы, подтверждающие начисление платы за ..."
97,Как подтвердить оплату ЖКХ при оформлении услу...,"Документы, подтверждающие начисление платы за ..."
98,Как подтвердить оплату ЖКХ при оформлении 583 ...,"Документы, подтверждающие начисление платы за ..."


In [40]:
data = dat_train.dropna()
data = data.reset_index()
del(data['index'])
data_quest = data[['QUESTION']]
data_answ = data[['ANSWER']]
data_quest = data_quest.to_numpy()
data_answ = data_answ.to_numpy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2729 entries, 0 to 2728
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   QUESTION  2729 non-null   object
 1   ANSWER    2729 non-null   object
dtypes: object(2)
memory usage: 42.8+ KB


In [41]:
n=0
examp_resp = {}
i = 0

while i < len(data_answ) - 1:
  if data_answ[i] == data_answ[i+1]:
    if str(n) not in examp_resp:
      examp_resp[str(n)] = {}
      examp_resp[str(n)]['examples'] = []
      examp_resp[str(n)]['responses'] = []
    examp_resp[str(n)]['examples'].append(str(data_quest[i]))
    examp_resp[str(n)]['responses'].append(str(data_answ[i]))
  else:
    if str(n) not in examp_resp:
      examp_resp[str(n)] = {}
      examp_resp[str(n)]['examples'] = []
      examp_resp[str(n)]['responses'] = []
    examp_resp[str(n)]['examples'].append(str(data_quest[i]))
    examp_resp[str(n)]['responses'].append(str(data_answ[i]))
    n+=1
  i+=1

i = 0
while i < len(examp_resp):
  examp_resp[str(i)]['responses'] = examp_resp[str(i)]['responses'][0]
  i+=1

In [42]:
dataset = []

for intent, intent_data in examp_resp.items():
    for example in intent_data['examples']:
      if len(dataset) >= 1000:
        break
      else:
        dataset.append([example, intent])

corpus = [text for text, intent in dataset]
y = [intent for text, intent in dataset]

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(2, 8))
X = vectorizer.fit_transform(corpus)

In [44]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression()

In [45]:
def get_intent(text):
    proba_list = clf.predict_proba(vectorizer.transform([text]))[0]
    max_proba = max(proba_list)
    if max_proba > 0.005:
      index = list(proba_list).index(max_proba)
      return clf.classes_[index]

In [46]:
def get_response_by_intent(intent):
    # phrases = examp_resp['intents'][intent]['responses']
    return examp_resp[intent]['responses']

In [47]:
import nltk

In [48]:
content = dataset
blocks = content

def clear_text(text):
    text = text.lower()
    alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя0123456789-I() '
    result = ''
    for c in text:
        if c in alphabet:
            result += c
    return result

dataset = []
questions = set()

for block in blocks:
    replicas = block
    if len(replicas) == 2:
        question = clear_text(replicas[0][2:])
        answer = replicas[1][2:]

        if question and answer and question not in questions:
            questions.add(question)
            dataset.append([question, answer])

In [49]:
search_dataset = {}
for question, answer in dataset:
    words = question.split(' ')
    for word in words:
        if word not in search_dataset:
            search_dataset[word] = []
        search_dataset[word].append((question, answer))

search_dataset = {
    word: word_dataset
    for word, word_dataset in search_dataset.items()
    if len(word_dataset) < 1000
}

In [50]:
def get_response_generatively(text):
    words = text.split(' ')

    words_dataset = set()
    for word in words:
        if word in search_dataset:
            words_dataset |= set(search_dataset[word])

    scores = []

    for question, answer in words_dataset:
        if abs(len(text) - len(question)) / len(question) < 0.4:
            distance = nltk.edit_distance(text, question)
            score = distance / len(question)
            if score < 0.4:
                scores.append([score, question, answer])

    if scores:
        return min(scores, key=lambda s: s[0])[2]

In [51]:
stats = {'intent': 0, 'generative': 0, 'fails': 0}

In [52]:
def bot(request):
    # NLU
    intent = get_intent(request)

    # Генерация ответа
    if intent:
        stats['intent'] += 1
        return get_response_by_intent(intent)

    response = get_response_generatively(request)
    if response:
        stats['generative'] += 1
        return response

    stats['fails'] += 1
    return 'некорректный запрос'

In [53]:
bot('как оформить карту?')

"['ребенок - гражданин РФ, имеющий место жительства (постоянную регистрацию) в СПб\\nИ\\nодин из родителей (усыновителей) или оба родителя (усыновителя) - граждане РФ, имеющие место жительства (постоянную регистрацию) в СПб;\\n\\nМать - встала на медицинский учет по поводу беременности в учреждении здравоохранения в срок до 20 недель (включительно)\\nИЛИ\\nребенка усыновили в возрасте до шести месяцев;\\n\\nОбращение последовало не позднее полутора лет со дня рождения ребенка.']"